In [ ]:
import random
import pickle

import numpy as np
import pandas as pd
from nltk.tokenize import RegexpTokenizer

from tensorflow.keras.models import Sequential, load_model
from tensorflow.keras.layers import LSTM, Dense, Activation
from tensorflow.keras.optimizers import RMSprop

In [ ]:
df = pd.read_csv('fake_or_real_news.csv')

In [ ]:
text = list(df.text.values)
joined_text = " ".join(text)

In [ ]:
len(joined_text)

29826765

In [ ]:
partial_text = joined_text[:100000]

In [ ]:
tokenizer = RegexpTokenizer(r"\w+")
tokens = tokenizer.tokenize(partial_text.lower())

In [ ]:
unique_tokens = np.unique(tokens)
unique_token_index = {token: idx for idx, token in enumerate(unique_tokens)}

In [ ]:
n_words = 10
inp_words = []
next_words = []

for i in range(len(tokens) - n_words):
  inp_words.append(tokens[i:i + n_words])
  next_words.append(tokens[i + n_words])

In [ ]:
X = np.zeros((len(inp_words), n_words, len(unique_tokens)), dtype=bool)
y = np.zeros((len(next_words), len(unique_tokens)), dtype=bool)

In [ ]:
for i,words in enumerate(inp_words):
  for j,word in enumerate(words):
    X[i,j, unique_token_index[word]] = 1
  y[i, unique_token_index[next_words[i]]] = 1

In [ ]:
model = Sequential()
model.add(LSTM(128, input_shape=(n_words, len(unique_tokens)), return_sequences=True))
model.add(LSTM(128))
model.add(Dense(len(unique_tokens)))
model.add(Activation("softmax"))

In [91]:
model.compile(loss='categorical_crossentropy', optimizer=RMSprop(learning_rate=0.01), metrics=['accuracy'])
model.fit(X,y, batch_size=128, epochs=20, shuffle=True)

Epoch 1/20
135/135 [==============================] - 53s 346ms/step - loss: 0.2324 - accuracy: 0.9731
Epoch 2/20
135/135 [==============================] - 45s 333ms/step - loss: 0.1803 - accuracy: 0.9803
Epoch 3/20
135/135 [==============================] - 47s 349ms/step - loss: 0.1471 - accuracy: 0.9833
Epoch 4/20
135/135 [==============================] - 44s 329ms/step - loss: 0.1386 - accuracy: 0.9825
Epoch 5/20
135/135 [==============================] - 46s 345ms/step - loss: 0.1351 - accuracy: 0.9825
Epoch 6/20
135/135 [==============================] - 42s 308ms/step - loss: 0.1139 - accuracy: 0.9859
Epoch 7/20
135/135 [==============================] - 44s 326ms/step - loss: 0.1168 - accuracy: 0.9825
Epoch 8/20
135/135 [==============================] - 44s 323ms/step - loss: 0.1074 - accuracy: 0.9851
Epoch 9/20
135/135 [==============================] - 44s 328ms/step - loss: 0.0991 - accuracy: 0.9864
Epoch 10/20
135/135 [==============================] - 44s 327ms/step - l

In [92]:
model.save("Mymodel.h5")

In [93]:
model = load_model("Mymodel.h5")

In [94]:
def predict_next_word(input_text, n_best):
  input_text = input_text.lower()
  X = np.zeros((1, n_words, len(unique_tokens)))
  for i, word in enumerate(input_text.split()):
    X[0, i, unique_token_index[word]] = 1

  predictions = model.predict(X)[0]
  return np.argpartition(predictions, -n_best)[-n_best:]

In [121]:
possible = predict_next_word("see this", 10)

1/1 [==============================] - 0s 31ms/step


In [122]:
possible

array([ 322,   98, 3217, 3616, 3601, 1649, 1741,  528, 1192, 2237])

In [123]:
print([unique_tokens[idx] for idx in possible])

['at', 'a', 'supports', 'who', 'well', 'in', 'is', 'but', 'even', 'now']


In [105]:
def generate_text(input_text, text_length, creativity=3):
  word_sequence = input_text.split()
  current = 0
  for _ in range(text_length):
    sub_seq = " ".join(tokenizer.tokenize(" ".join(word_sequence).lower())[current:current+ n_words])
    try:
      choice = unique_tokens[random.choice(predict_next_word(sub_seq, creativity))]
    except:
      choice = random.choice(unique_tokens)
    word_sequence.append(choice)
    current += 1
  return " ".join(word_sequence)

In [107]:
generate_text("He is trying to understand that",100, 1)


1/1 [==============================] - 0s 29ms/step


'He is trying to understand that kgb action door sanctions how we we years such problems trump it trump men long both leaders a a a a all little it we we many we we with it we we we we we for we we details ayotte this in trump her and because we we we we we we home it it we we we we supporting we we we with and we we we we we with it it we we we we supporting we we we with and we we we we we with it it we we we we supporting we we we'